In [1]:
! pip install transformers
! pip install datasets

     |████████████████████████████████| 2.8 MB 3.6 MB/s 
     |████████████████████████████████| 50 kB 7.4 MB/s 
     |████████████████████████████████| 895 kB 56.9 MB/s 
     |████████████████████████████████| 636 kB 64.9 MB/s 
     |████████████████████████████████| 3.3 MB 57.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 4.1 MB/s 
     |████████████████████████████████| 243 kB 60.3 MB/s 
     |████████████████████████████████| 119 kB 62.2 MB/s 


In [2]:
cd '/content/drive/MyDrive/CAMemBERT2'

/content/drive/MyDrive/CAMemBERT2


In [3]:
from transformers import TrainingArguments,Trainer,AutoModelForSequenceClassification,AutoModel
import numpy as np
from math import floor
import torch
from torch.cuda import is_available
from collections import defaultdict
from scipy.stats import spearmanr
from sklearn.metrics import cohen_kappa_score,classification_report,fbeta_score
from models_f import MultiTaskModel
from trainer_f import MultiTaskModelTrainer
from trainer_utils import (training_args,training_kwargs,get_dataset,
                           freeze_layers,normalise_scores,create_params_grid)

In [4]:
for pretrained_model_name in ['distilroberta-base']:
    model_type = 'roberta' if 'roberta' in pretrained_model_name else 'bert'
    model_n_layers = 6 if 'roberta' in pretrained_model_name else 12
    params_grid = create_params_grid({'tasks':[['aes','ged','ner']],
                                      'lr':[4e-5],
                                      'shared_encoder_n_layers':[0],
                                      'frozen_layers':[None],
                                      'output_layer':[-1],
                                      'init_task_weightings':[x*0.1 for x in range(1,10)]
                                      }
                                      )

    prev_task = None
    for i,params in enumerate(params_grid):
        print(params)
        if params['tasks']!=prev_task:
            dataset_obj,dataset_dict = get_dataset(params['tasks'],pretrained_model=pretrained_model_name,max_length=512)
            dataset_dict = dataset_dict.map(normalise_scores)

        trainer_kwargs,model_kwargs = training_kwargs(
            tasks=params['tasks'],
            init_task_weightings = params['init_task_weightings'],
            class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
            early_stopping_patience=2,
            shared_encoder_n_layers=params['shared_encoder_n_layers'],
            output_layer = {'aes':-1,
                            'ged':params['output_layer'],
                            'ner':params['output_layer']},
            early_stopping_metric='los',
            normalised_values=[40,0],
            frozen_layers=params['frozen_layers'],
            save_results=True)

        args = training_args(lr=params['lr'],epochs=8)
        model = MultiTaskModel(kwargs_dict = model_kwargs,pretrained_model=pretrained_model_name)
        model = freeze_layers(model,frozen_layers=params['frozen_layers'],model_type=model_type)
        dev_set = 'dev' if params['tasks']!='ner' else 'validation'
        trainer = MultiTaskModelTrainer(
            model,
            args,
            train_dataset=dataset_dict['train'],
            eval_dataset=dataset_dict[dev_set],
        )
        trainer.kwargs_dict = trainer_kwargs
        trainer.test_dataset = dataset_dict['test']
        trainer.train()
        prev_task=params['tasks']

{'frozen_layers': None, 'init_task_weightings': 0.1, 'lr': 4e-05, 'output_layer': -1, 'shared_encoder_n_layers': 0, 'tasks': ['aes', 'ged', 'ner']}


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

{'frozen_layers': None, 'init_task_weightings': 0.9, 'lr': 4e-05, 'output_layer': -1, 'shared_encoder_n_layers': 0, 'tasks': ['aes', 'ged', 'ner']}


loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab

Step,Training Loss
500,0.045400
1000,0.030200
1500,0.026200


aes_weight_coef : 0.8999999761581421
ged_weight_coef : 0.10000000149011612
ner_weight_coef : 0.009999999776482582

losses {'aes': 0.0029029153330728066, 'ged': 0.011077113882746807, 'ner': 0.003204465248123851, 'total_loss': 0.017184494463943465}

aes_metrics {'rmse_aes': 4.983305462575351, 'pearson_aes': 0.5203488235516072, 'spearman_aes': 0.5393420501919746, 'kappa_aes': 0.4733276385165984}

ged               precision    recall  f1-score   support

           0       0.94      0.78      0.85     25174
           1       0.33      0.69      0.44      3917

    accuracy                           0.77     29091
   macro avg       0.63      0.74      0.65     29091
weighted avg       0.86      0.77      0.80     29091

ged_metrics {'accuracy_ged': 0.7663882300367811, 'f1_score_macro_ged': 0.6479583208870832, 'f1_score_weighted_ged': 0.7971590900920337, 'f_0_5_ged': 0.5655221327548083}

ner               precision    recall  f1-score   support

           0  0.943959404 0.996258098 0.969



Training completed. Do not forget to share your model on huggingface.co/models =)


